
The NQueen problem:\
As prof.BabaAli has taught in class, in order to solve any problem with a genetic algorithm we must first find an optimal way to show the answers to the problem as chromosomes.\
As discussed in class, we define the chromosomes as a $1$-dimensional array of length n (assuming that the board is $n*n$). This is because we can be sure that when we want to place $n$ queens in an $n$ by $n$ table, since there can only and only be a single queen in each column (note that the count of the queens and the size of the table both match - $n$). As a result, we shall only have one metric for each column's queen: what row that queen is located in.\
so we know what our chromosomes look like. now we should think about a fitness function.


As we also have discussed in class, we define the fitness function as the numbre of collisions a formation of queens have. meaning that if we put all the queens in a row, then the number of collisions will be $C(n, 2)$. However, we want to make the fitness give a higher score when the solution is more optimal, thus we invert the asnwers by substracting them from the +inf. we can also define +inf as the maximum score available in the problem (with some extra addition for good measure).\
As a result, the fitness function will be:\
\begin{equation}
fitness(chromosome) = maximumPossibleFitness - number Of Collisions (chromosome)
\end{equation}

Thus, when simplified:
\begin{equation}
fitness(chromosome) = \\+ (C(n, 2) + n) \\- row Collisions (chromosome) - column Collisions (chromosome) - diagonal Collisions (chromosome)
\end{equation}

But, we also from the definition of chromosome we know that the column collisions are ZERO (since there is exactly one queen in each column). So the new formula will be:
\begin{equation}
fitness(chromosome) = (C(n, 2) + n) - row Collisions (chromosome) - diagonal Collisions (chromosome)
\end{equation}

In [ ]:
import sys #to have access to stderr

def fitness (chromosome):
  """
  calculate a fitness value for a given chromosome
  """
  n = len(chromosome)
  #calculating row collisions:
  rcollide = 0
  for i in range(n):
    for j in range(i+1, n):
      if (chromosome[i] == chromosome[j]):
        rcollide += 1
  #print("rcollide is ", rcollide, file=sys.stderr)

  #calculating diagonal collisions:
  dcollide = 0
  diagonal = [0 for _ in range(2*n-1)]
    #main diagonal:
  for i in range(n):
    dcollide += diagonal[n - chromosome[i] + i - 1]
    diagonal[n - chromosome[i] + i - 1] += 1
    #secondary diagonal:
  diagonal = [0 for _ in range(2*n-1)]
  for i in range(n):
    dcollide += diagonal[chromosome[i] + i]
    diagonal[chromosome[i] + i] += 1
  #print("dcollide is ", dcollide, file=sys.stderr)
  
  #calculating the inf
  infinity = ((n+1)*(n)) // 2
  #print("infinity is ", infinity, file=sys.stderr)

  return infinity - rcollide - dcollide

We now define the crossover:\
In order to do crossover, we stick to the solution we found in the class. First choose 2 parents and give as an input, then choose a random position such as i, and make child by using the first untill i-th position of the first parent and i+1-th untill the end of the second parent.
This is one of the ways to do crossover between two arrays that might be permutations and since it's decent and it works for out problem.

In [ ]:
import random

def crossover(parent_one, parent_two):
  n = len(parent_one)
  pos_i = random.randint(0, n-1)
  child = []
  for j in range(pos_i+1):
    child.append(parent_one[j])
  for j in range(pos_i+1, n):
    child.append(parent_two[j])
  return child

#print("testing the crossover: ", crossover([1,2,3,4,5,6,7,8], [8,7,6,5,4,3,2,1]), file=sys.stderr)

We should also define mutation. The easiest solution that comes to mind is to choose two queens and swap their columns (because the solution of the problem will be a chromosome which is a permutation of 0 to n-1). However, while we are searching for the answer, there is no guarantee that the current chromosome is a permutation. Thus we code the permutation to change a random queens position to a random position on its column.

In [ ]:
def mutate(chromosome):
  n = len(chromosome)
  pos_column = random.randint(0, n-1)
  pos_row_new = random.randint(0, n-1)
  chromosome[pos_column] = pos_row_new
  return chromosome

#print("testing the permutation: ", mutate([1,2,3,4,5,6,7,8]), file=sys.stderr)

Finally, we should now start initializing the population. In order to do so, we make a population of random permutations using random shuffle function available in python. \\

---

WE CAN ALSO SET THE POPULATION SIZE AND THE NUMBER OF QUEENS FROM THIS PART OF THE CODE:


In [ ]:
POP_SIZE = 100
NUMBER_OF_QUEENS = 100

population = [[i for i in range(NUMBER_OF_QUEENS)] for _ in range(POP_SIZE)]

for i in range(POP_SIZE):
  random.shuffle(population[i])

#print("population is:", *population, file=sys.stderr, sep="\n")

We should now define the steps we need to take in each generation (Or if you are familiar with ML terms, we should define what steps to take in each epoch). We use random selection of parents as it is good balance between exploitation and exploration, we then choose the 90-best chromosomes based on their fitness and fill the rest of the population from the losing chromosomes randomly.
In order to have a fair compromise between exploration and exploitaiton, I decided to set a variable mutation probablity to encourage muatations early on, and set it to be a constant 10% later on. However, still set a random 90% chance every 1000 steps to help avoid local extermums.

In [ ]:
def generation(population, mutate_prob):
  """
  takes the popluation, and makes a new generation using the functions defined above
  """
  next_generation = []
  fitness_mem = [fitness(chromosome) for chromosome in population]
  fitness_sum = sum(fitness_mem)
  for i in range(POP_SIZE):
    parent_one_probablity = random.randint(1, fitness_sum)
    parent_two_probablity = random.randint(1, fitness_sum)
    
    sum_prob = 0
    parent_one = parent_two = -1
    for i in range(POP_SIZE):
      sum_prob += fitness_mem[i]
      if (parent_one == -1 and sum_prob >= parent_one_probablity):
        parent_one = i
      if (parent_two == -1 and sum_prob >= parent_two_probablity):
        parent_two = i
      new_child = crossover(population[parent_one], population[parent_two])
      mutate_rand = random.random()
      if mutate_rand <= mutate_prob:
        new_child = mutate(new_child)
      next_generation.append(new_child)
    
    population += next_generation
    next_generation = []
    population.sort(key=fitness)
    rand_amount = POP_SIZE // 10
    for _ in range(rand_amount):
      next_generation.append(population[random.randint(0, POP_SIZE+rand_amount-1)])
    next_generation = next_generation + population[POP_SIZE+rand_amount:]
    return next_generation

gen_count = 1
mutate_prob = 0.90
while True:
  if (gen_count % 1000 == 0):
    print("on generation: ", gen_count, " max fit: ", fitness(population[-1]))
    if mutate_prob > 0.10:  
      mutate_prob -= 0.10
    population = generation(population, 0.90)
  else:
    population = generation(population, mutate_prob)
  ans_fit = ((NUMBER_OF_QUEENS+1) * NUMBER_OF_QUEENS)//2
  if (fitness(population[-1]) == ans_fit):
    print("on generation: ", gen_count, " ans fit: ", fitness(population[-1]))
    print (population[-1])
    break
  gen_count += 1

on generation:  1000  max fit:  5049
on generation:  1870  ans fit:  5050
[66, 16, 33, 44, 75, 87, 12, 63, 42, 29, 71, 84, 74, 21, 41, 39, 95, 98, 96, 64, 25, 47, 52, 86, 13, 72, 91, 3, 30, 27, 2, 49, 35, 15, 94, 28, 23, 76, 38, 55, 11, 93, 80, 34, 56, 24, 79, 31, 1, 59, 73, 65, 19, 97, 50, 18, 88, 85, 32, 99, 61, 0, 20, 22, 8, 53, 60, 57, 37, 90, 83, 4, 89, 68, 78, 26, 51, 54, 9, 17, 40, 67, 48, 82, 45, 92, 69, 6, 10, 81, 70, 46, 77, 58, 62, 7, 14, 43, 5, 36]
